# A small-scale version of our pipeline

### Imports

In [28]:
import pandas as pd
import regex as re
from cleantext import clean
import pyarrow.feather as feather
from multiprocessing import Pool
import gc
import nltk
import itertools
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import *
import time
from collections import Counter

from sklearn.model_selection import train_test_split    # splitting the data 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

### Loading the dataset

In [2]:
import pandas as pd
import regex as re
from cleantext import clean
raw_data = pd.read_csv("data/sample_preprocessed_ver_2023-03-19-1679240881")

/var/folders/gj/r8x5s5mn7jg12hs3c28wpg4c0000gn/T/ipykernel_21707/3660192515.py:4: DtypeWarning: Columns (1,2,3,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv("data/sample_preprocessed_no_punct.csv")


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,0.0,11467794,8027,9787368,nytimes.com,reliable,https://query.nytimes.com/gst/fullpage.html?re...,"['kashaaudrey', 'sission', 'lifelong', 'new', ...",2018-02-11 00:48:58.787555,2018-02-11 00:14:20.346838,2018-02-11 00:14:20.346871,"Paid Notice: Deaths KASHA, AUDREY SISSION",NaN,NaN,['KASHA AUDREY SISSION'],"KASHA--Audrey Sission, a lifelong New Yorker, ...",NaN,NaN,nytimes
1,1.0,4591569,8760,3009581,thinkprogress.org,political,https://thinkprogress.org/into-the-valley-of-d...,"['forward', 'light', 'brigadewa', 'man', 'dism...",2017-11-18T20:01:27.400599,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,"Into The Valley Of Death Rode The 600, Into Th...",NaN,NaN,[''],NaN,"#Climate Change, #Climate",NaN,NaN
2,2.0,6162754,2521,4251195,truthandaction.org,bias,http://www.truthandaction.org/woman-thrown-off...,"['woman', 'thrown', 'plane', 'said', 'hillari'...",2017-11-27T01:15:32.269834,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Woman Thrown Off Plane When She Said Hillary i...,NaN,NaN,[''],NaN,NaN,NaN,NaN
3,3.0,2542246,2019,1769246,ecowatch.com,political,https://www.ecowatch.com/cuban-province-well-o...,"['cuban', 'provinc', 'well', 'way', 'num', 're...",2017-11-10T11:18:44.524042,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Cuban Province Well on Its Way to 100% Renewab...,"Guest Contributor, Sierra Club, Common Dreams,...",NaN,"['featured', 'renewables', 'business', 'cuba']",President Obama’s recent announcement that he ...,NaN,NaN,NaN
4,4.0,3753783,4806,2429386,weeklystandard.com,political,http://www.weeklystandard.com/print/the-times-...,"['new', 'york', 'time', 'greet', 'deleg', 'fro...",2017-11-13T18:09:27.760857,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,The Times Repeats Itself,To The Scrapbook,NaN,['The Scrapbook'],The New York Times greeted delegates with a fr...,NaN,NaN,NaN


In [16]:
raw_data = raw_data.sample(frac=0.1, random_state=0)

In [3]:
# convert the strings to lists
pattern = re.compile(r"\w+")
def string_to_list(s):
    return pattern.findall(s)
raw_data['content'] = raw_data['content'].apply(string_to_list)

In [17]:
len(raw_data)

65001

In [5]:
def fix_labels(df): 
    df.type = df.type.replace({'political': 'reliable', 'junksci': 'fake', 'bias' : 'fake', 'satire': 'fake', 'conspiracy': 'fake', 'rumor': 'fake', 'unreliable' : 'fake', 'clickbait': 'fake', 'hate': 'fake'})
    df = df[df.type != 'unknown']
    df = df[df.type != 'type']
    df = df[df.type.notnull()]
    return df

In [6]:
def labelperc(df): 
    labeldict = {}
    for i in df.type: 
        if i in labeldict: 
            labeldict[i] +=1
        else: 
            labeldict[i] = 1
    for i in labeldict: 
        labeldict[i] = labeldict[i]/len(df)*100
    return labeldict

In [18]:
data = fix_labels(raw_data)

In [19]:
print(labelperc(data))

{'reliable': 50.93170829975826, 'fake': 49.06829170024174}


In [20]:
data.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
327596,327590.0,11227715,7948,9547203,nytimes.com,reliable,https://www.nytimes.com/2013/01/15/us/crime-ri...,"[nearli, progress, eras, last, year, san, bern...",2018-02-11 00:46:34.077532,2018-02-11 00:14:20.346838,2018-02-11 00:14:20.346871,"A Poorer San Bernardino, and a More Dangerous ...",Ian Lovett,NaN,"['San Bernardino County (Calif)', 'Bankruptcie...","Officials in San Bernardino, Calif., were maki...",NaN,NaN,nytimes
531597,531587.0,4454623,1814,2850366,wikispooks.com,fake,https://wikispooks.com/wiki/Special:Browse/Tha...,"[queri, depth, predefin, properti, inform, dep...",2017-11-18T20:01:27.400599,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Thales UK,NaN,NaN,[''],NaN,NaN,NaN,NaN
506960,506950.0,8135299,1705,6724283,nationalreview.com,reliable,http://www.nationalreview.com/node/152734/%E2%...,"[plu, one, articl, googl, plu, thank, ali, alf...",2017-11-27T01:14:42.983556,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Iran News Round Up,NaN,NaN,"['National Review', 'National Review Online', ...",NaN,NaN,NaN,NaN
297286,297281.0,3934676,841,2556135,beforeitsnews.com,fake,http://beforeitsnews.com/environment/2011/09/e...,"[news, green, news, 4u, empir, state, build, c...",2017-11-13T18:09:27.760857,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Empire State Building Achieves LEED Gold,Green News,NaN,[''],NaN,NaN,NaN,NaN
227686,227682.0,1949789,9457,1179853,express.co.uk,fake,https://www.express.co.uk/life-style/health/84...,"[getti, rheumatoid, arthriti, patient, shingl,...",2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Arthritis treatment news: THIS vaccine should ...,Lauren Clark,NaN,[''],ARTHRITIS patients have a higher risk of devel...,NaN,NaN,NaN


In [21]:
print(labelperc(data))

{'reliable': 50.93170829975826, 'fake': 49.06829170024174}


### Simple Model - Logistic Regression

In [22]:
X_train, X_val, y_train, y_val = train_test_split(data['content'], data['type'], test_size=0.2, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=0)

In [23]:
vectorizer = CountVectorizer(analyzer=lambda x : x)
vectorizer.fit(X_train)

CountVectorizer(analyzer=<function <lambda> at 0x62b1f1d80>)

In [24]:
X_train = vectorizer.transform(X_train)
X_val = vectorizer.transform(X_val)
X_test = vectorizer.transform(X_test)

In [25]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [26]:
pred = model.predict(X_val)

In [29]:
accuracy_score(y_val, pred)

0.8494208494208494